# Book Data Analysis and Neo4j Integration

## Author: Yousuf Rajput, Tarek Tarif

## Overview

This Jupyter notebook performs an analysis on book-related data and integrates it with a Neo4j graph database. The analysis includes consolidating and reordering book data, reading and processing JSON and CSV files, and creating relationships between various entities like Books, Authors, Ratings, Languages, and Reviews.

## Code Sections

1. **Consolidate and Reorder CSV Data:**
   - Reads and consolidates book data from multiple CSV files.
   - Reorders the data to ensure a common structure.

2. **JSON to CSV Conversion:**
   - Reads a JSON file in chunks and converts it to a CSV file.

3. **Remove Columns from CSV:**
   - Removes specified columns from a CSV file.

4. **Extract Common ISBNs and Save to new CSV file:**
   - Finds common ISBNS between 2 datasets and saves to CSV file.

5. **Correct CSV files:**
   - Correct syntax errors in CSV files for proper instertion.

6. **Text Extraction from 1000 Books Page:**
   - Scrapes text content for Book titles and saves it to a text file.

7. **Read Wikipedia Tables:**
   - Reads tables from a Wikipedia page and saves the 'Book' column to text files.

8. **Neo4j Database Integration:**
   - Connects to a Neo4j database.
   - Inserts structured and semi-structured data into the database.
   - Creates relationships between nodes (Books, Ratings, Languages, Authors, Reviews).

9. **Neo4j Queries for Analysis:**
   - Executes 5 complex Neo4j queries.

## Requirements

- Ensure Neo4j is installed and a database is running.
- Python libraries required: pandas, requests, beautifulsoup4, py2neo.

### Required Data:
- Semi-Structured Dataset Download Link: https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/goodreads_books.json.gz
- Structured Dataset Download Link: https://www.kaggle.com/datasets/bahramjannesarr/goodreads-book-datasets-10m/data

**Note:** Update Neo4j connection details and file paths as needed.


In [38]:
# Import required libraries
import pandas as pd
import json
import csv
import ast
import os
from bs4 import BeautifulSoup
import requests
import re
from py2neo import Graph

# Step 1: Consolidate and Reorder CSV Files

This step defines a function `consolidate_and_reorder` that consolidates and reorders data from multiple CSV files into a single CSV file. It uses the pandas library for data manipulation.

### Function Signature
```python
def consolidate_and_reorder(book_files, output_file):
    """
    Consolidate and reorder data from multiple CSV files.

    Parameters:
    - book_files (list): List of CSV file paths to consolidate.
    - output_file (str): Output file path to save the consolidated data.

    Returns:
    None
    """


In [1]:
def consolidate_and_reorder(book_files, output_file):
    common_header = None
    consolidated_data = []

    for file_path in book_files:
        # Read the first row (header) of the CSV file
        header = pd.read_csv(file_path, nrows=0).columns.tolist()

        # Set as common header if not set yet
        if common_header is None:
            common_header = header
        else:
            # Ensure that the current header matches the common header
            if header != common_header:
                # Find columns that are present in the common header but not in the current header
                missing_columns_common = [col for col in common_header if col not in header]

                # Find columns that are present in the current header but not in the common header
                missing_columns_current = [col for col in header if col not in common_header]

                # Drop the missing columns from both the common header and the current header
                common_header = [col for col in common_header if col not in missing_columns_common]
                header = [col for col in header if col not in missing_columns_current]

                # Warn user about the missing columns
                print(f"Warning: Columns {missing_columns_common} are missing in file '{file_path}' and will be dropped.")

        # Read the entire CSV file and append to consolidated_data
        data = pd.read_csv(file_path, usecols=common_header)
        consolidated_data.append(data)

    # Concatenate all data frames
    consolidated_df = pd.concat(consolidated_data, ignore_index=True)

    # Reorder columns to match the common header
    consolidated_df = consolidated_df[common_header]

    # Save the consolidated and reordered data to 'books.csv'
    consolidated_df.to_csv(output_file, index=False)

# Input book files
book_files = [
    'archive/book1-100k.csv',
    'archive/book100k-200k.csv',
    'archive/book200k-300k.csv',
    'archive/book300k-400k.csv',
    'archive/book400k-500k.csv',
    'archive/book500k-600k.csv',
    'archive/book600k-700k.csv',
    'archive/book700k-800k.csv',
    'archive/book800k-900k.csv',
    'archive/book900k-1000k.csv',
    'archive/book1000k-1100k.csv',
    'archive/book1100k-1200k.csv',
    'archive/book1200k-1300k.csv',
    'archive/book1300k-1400k.csv'
]

# Output file
output_file = 'books.csv'

# Consolidate and reorder the data
consolidate_and_reorder(book_files, output_file)

print(f"Consolidated data saved to '{output_file}'.")

Consolidated data saved to 'books.csv'.


# Step 2: Read JSON File in Chunks and Save to CSV

The function `count_entries` counts the number of entries in a JSON file. It reads the JSON file and uses the `json.load` function to parse the data. The count of entries is then returned. The JSON file is then loaded in chunks to a csv file for faster processing. 


### Function Signature
```python
def count_entries(json_file):
    """
    Count the number of entries in a JSON file.

    Parameters:
    - json_file (str): Path to the JSON file.

    Returns:
    int: Number of entries in the JSON file.
    """


```



In [2]:
def count_entries(json_file):
    with open(json_file, 'r') as f:
        entries = 0
        for line in f:
            try:
                # Try to load each line as a JSON object
                json_object = json.loads(line)
                entries += 1
            except json.JSONDecodeError:
                # Handle invalid JSON on a line (skip or log as needed)
                print(f"Skipping invalid JSON on line: {line.strip()}")
    return entries

json_file = 'proj/Data/Semi-Structured/OLD/goodreads_books.json'  # replace with your file path
num_entries = count_entries(json_file)
print(f'The JSON file has {num_entries} entries.')


The JSON file has 2360655 entries.


In [3]:
# Read the JSON file in chunks
json_reader = pd.read_json(json_file, lines=True, chunksize=25000)

# Open the CSV file
with open('goodreads_books.csv', 'w', encoding='utf-8') as f:
    # Initialize the chunk counter
    chunk_counter = 0

    # Process each chunk
    for chunk in json_reader:
        # Increment the chunk counter
        chunk_counter += 1

        # Write the chunk to the CSV file
        chunk.to_csv(f, index=False, header=f.tell()==0)

        # Print the chunk counter
        print(f'Processed {chunk_counter} chunks')
print('Done')

Processed 1 chunks
Processed 2 chunks
Processed 3 chunks
Processed 4 chunks
Processed 5 chunks
Processed 6 chunks
Processed 7 chunks
Processed 8 chunks
Processed 9 chunks
Processed 10 chunks
Processed 11 chunks
Processed 12 chunks
Processed 13 chunks
Processed 14 chunks
Processed 15 chunks
Processed 16 chunks
Processed 17 chunks
Processed 18 chunks
Processed 19 chunks
Processed 20 chunks
Processed 21 chunks
Processed 22 chunks
Processed 23 chunks
Processed 24 chunks
Processed 25 chunks
Processed 26 chunks
Processed 27 chunks
Processed 28 chunks
Processed 29 chunks
Processed 30 chunks
Processed 31 chunks
Processed 32 chunks
Processed 33 chunks
Processed 34 chunks
Processed 35 chunks
Processed 36 chunks
Processed 37 chunks
Processed 38 chunks
Processed 39 chunks
Processed 40 chunks
Processed 41 chunks
Processed 42 chunks
Processed 43 chunks
Processed 44 chunks
Processed 45 chunks
Processed 46 chunks
Processed 47 chunks
Processed 48 chunks
Processed 49 chunks
Processed 50 chunks
Processed

# Step 3: Remove Specified Columns from CSV File

Columns that are not needed are removed from the CSV file (`goodreads_books.csv`) and the updated data is saved to a new CSV file (`goodreads_books_updated.csv`).

### Steps:
1. Define a list of columns to be removed (`columns_to_remove`).
2. Create a new CSV file (`goodreads_books_updated.csv`) to save the updated data.
3. Read the original CSV file (`goodreads_books.csv`) in chunks to manage memory efficiently.
4. Iterate through each chunk, remove specified columns, and append the updated chunk to the new CSV file.
5. Print progress information during chunk processing.
6. Display 'Done!' once the entire CSV file has been processed.


In [4]:
# List of columns to be removed
columns_to_remove = ['text_reviews_count', 'series', 'description', 'link', 'num_pages', 'isbn13', 'url', 'authors', 'image_url']

# Create a new CSV file to save the updated data
with open('goodreads_books_updated.csv', 'w') as f:
    pass

# Read the CSV file in chunks
chunksize = 10 ** 4  # adjust this value depending on performance
chunk_counter = 0
for chunk in pd.read_csv('goodreads_books.csv', chunksize=chunksize):
    chunk_counter += 1
    print(f'Processing chunk {chunk_counter}...')
    # Remove the columns
    chunk = chunk.drop(columns_to_remove, axis=1)
    # Append the updated chunk to the new CSV file
    chunk.to_csv('goodreads_books_updated.csv', mode='a', index=False)
print('Done!')

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...
Processing chunk 29...
Processing chunk 30...
Processing chunk 31...
Processing chunk 32...
Processing chunk 33...
Processing chunk 34...
Processing chunk 35...
Processing chunk 36...
Processing chunk 37...
Processing chunk 38...
Processing chunk 39...
Processing chunk 40...
Processing chunk 41...
Processing chunk 42...
Processing chunk 43...
Processing chunk 44.

# Step 4: Extract Common ISBNs and Save to a New CSV File

Extract common ISBNs from the 'goodreads_books_updated.csv' file that match those in the 'books.csv' file. The resulting data is then saved to a new CSV file named 'goodreads_books_common.csv'. This process helps filter and retain only the entries with common ISBNs.

### Steps:
1. Read the 'books.csv' file and store the unique ISBNs in a set.
2. Create a new CSV file ('goodreads_books_common.csv') to save the updated data.
3. Read the 'goodreads_books_updated.csv' file in chunks for efficient memory usage.
4. Remove rows with 'null' ISBN values from each chunk.
5. Identify common ISBNs between the chunk and the 'books.csv' file.
6. Filter the chunk to include only rows with common ISBNs.
7. Append the updated chunk to the new CSV file.


In [5]:
# List of common ISBNs
common_isbns = []

# Read the 'books.csv' file and store the ISBNs
df_books = pd.read_csv('books.csv')
book_isbns = set(df_books['ISBN'])

# Create a new CSV file to save the updated data
with open('goodreads_books_common.csv', 'w') as f:
    pass

chunksize = 10 ** 4  # adjust this value depending on performance
for chunk in pd.read_csv('goodreads_books_updated.csv', chunksize=chunksize):
    # Remove rows with 'null' ISBN/title values
    chunk = chunk[~chunk['isbn'].isnull()]
    chunk = chunk[~chunk['title'].isnull()]
    # Find the common ISBNs
    common_isbns_chunk = pd.Series(list(set(chunk['isbn']).intersection(book_isbns)))
    common_isbns.extend(common_isbns_chunk)

    # Filter the chunk to only include rows with common ISBNs
    chunk = chunk[chunk['isbn'].isin(common_isbns_chunk)]

    # Append the updated chunk to the new CSV file
    chunk.to_csv('goodreads_books_common.csv', mode='a', index=False)
print('Done!')

Done!


# Step 5: Correct CSV File with Quotes

Takes input a CSV file named 'goodreads_books_common.csv', corrects potential problems with quotes in the data, and saves the corrected data to a new CSV file named 'fixed_goodreads_books_common.csv'.

### Steps:
1. Define a function `escape_quotes` that uses `ast.literal_eval` to handle escaping quotes in a given value.
2. Specify input and output file paths.
3. Open the input CSV file in read mode and the output CSV file in write mode with newline='' to prevent extra newline characters.
4. Initialize CSV readers and writers.
5. Iterate through rows in the input CSV file.
6. Apply the `escape_quotes` function to all values in each row.
7. Write the corrected row to the output CSV file.
8. Print a message indicating the completion of the correction process.


In [6]:
def escape_quotes(value):
    try:
        return ast.literal_eval(f'"{value}"')
    except (SyntaxError, ValueError):
        return value

input_file = 'goodreads_books_common.csv'
output_file = 'fixed_goodreads_books_common.csv'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.DictReader(infile)
    
    # Check if the CSV file has a header
    fieldnames = reader.fieldnames if reader.fieldnames is not None else ['column1', 'column2', 'column3']  # Replace with actual column names
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    # Write the header only if it exists
    if fieldnames is not None:
        writer.writeheader()

    for row in reader:
        # Apply the escape_quotes function to all values in the row
        row = {key: escape_quotes(value) for key, value in row.items()}
        # Write the corrected row to the output file
        writer.writerow(row)
        
os.rename(output_file, input_file)
print(f"Correction complete. Fixed data written to {input_file}")


Correction complete. Fixed data written to goodreads_books_common.csv


In [7]:
#Load CSV, Get Sample, and Save
df = pd.read_csv('goodreads_books_common.csv')
df = df.head(100)
df.to_csv('goodreads_common_sample.csv', index=False)

# Step 6: Extract Titles from Webpage and Write to Text File

This code retrieves a list of book titles from a specific webpage using a provided URL. It utilizes BeautifulSoup and CSS selectors to locate the relevant HTML element containing the book titles. The extracted titles are then processed and written to a text file named '1000books.txt'. If the specified HTML element is not found, an appropriate message is printed.


In [8]:
# URL of the webpage you want to extract
url = "https://libraryof1000books.wordpress.com/the-list-of-1000-books/"

# CSS selector for the specific ol element
selector = "#post-285 > div > ol:nth-child(2)"

# Send a GET request
response = requests.get(url)

# Parse the content of the request with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the specific ol element using the selector
specific_ol = soup.select_one(selector)

# Extract text from the ol element
if specific_ol:
    # Extract only the titles from list items
    titles = [re.split(r'\s*–\s*', item.get_text(strip=True))[0] for item in specific_ol.find_all('li')]

    # Join the titles into a single string
    titles_text = '\n'.join(titles)

    # Write the extracted titles to '1000books.txt'
    with open('1000books.txt', 'w', encoding='utf-8') as f:
        f.write(titles_text)
else:
    print("Element not found.")


# Step 7: Combine Best-Selling Books from Wikipedia Tables

This code retrieves tables from a Wikipedia page containing information about best-selling books. The tables are read into Pandas DataFrames using the provided URL. The contents of the 'Book' column from these tables are then combined and written to a text file named 'topSellingBooks.txt'. The final output is a consolidated list of best-selling books.


In [9]:

url = 'https://en.wikipedia.org/wiki/List_of_best-selling_books#More_than_100_million_copies'
tables = pd.read_html(url)

# Combine the contents of the files into a single file
with open('topSellingBooks.txt', 'w', encoding='utf-8') as outfile:
    for table in tables:
        # If the table has a 'Book' column
        if 'Book' in table.columns:
            # Extract the 'Book' column
            books = table['Book']
            # Write the contents of the 'Book' column to the output file
            outfile.write('\n'.join(books))

print("Contents of the files combined into 'topSellingBooks.txt'")

Contents of the files combined into 'topSellingBooks.txt'


# Step 8: Connect to Neo4j database

In [17]:
# Connect to the Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))  # replace with your actual username and password

# Data Import and Indexing for Neo4j Database

### Query 1: Adding structured data to Neo4j DB
This query loads data from a CSV file (`books.csv`) and creates nodes in Neo4j, setting properties based on the CSV data.

### Query 2: Creating index on ISBN
A Neo4j index is created on the 'ISBN' property of the 'Book' nodes to improve query performance.

### Query 3: Importing semi-structured data into Neo4j
Data from a CSV file (`goodreads_books_common.csv`) is iteratively imported into Neo4j, updating existing 'Book' nodes based on ISBN.

### Query 4: Creating Index on 'Name' for faster runtime
Indexes are created for 'Name' property of 'Book', 'name' property of 'Author', and 'count' property of 'Review'.

### Query 5: Setting all values of best_seller to false
This query iterates over all 'Book' nodes, setting the 'best_seller' and 'in_1000_collection' properties to false.

### Query 6: Merging nodes with duplicate titles
Nodes with duplicate 'Name' properties are merged, and extra nodes are detached from the database.

### Set true queries
Values are set to true for specific conditions using queries `qtrue` and `qtrue2`, based on book names from external text files.



In [11]:
# Query 1: Adding structured data to Neo4j DB
insertSD = """
CALL apoc.periodic.iterate(
  'LOAD CSV WITH HEADERS FROM "file:///books.csv" AS row RETURN row',
  'CREATE (b:Book) SET b = row',
  {batchSize:1000, iterateList:true, parallel:false}
)
"""
graph.run(insertSD)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
702,701789,18,701789,0,0,0,{},"{total: 702, errors: {}, committed: 702, failed: 0}","{total: 701789, errors: {}, committed: 701789, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 0, nodesDeleted: 0, nodesCreated: 701789, labelsRemoved: 0, labelsAdded: 701789, propertiesSet: 12632201}"


In [12]:
# Query 2: Creating index on ISBN
isbnIndex = """
CREATE INDEX FOR (b:Book) ON (b.ISBN)
"""
graph.run(isbnIndex)

(No data)

In [18]:
# Query 3: Importing semi-structured data into Neo4j
insertSSD = """
CALL apoc.periodic.iterate(
  'LOAD CSV WITH HEADERS FROM "file:///goodreads_books_common.csv" AS row RETURN row',
  'MATCH (b:Book {ISBN: row.isbn}) SET b += row',
  {batchSize:1000, iterateList:true, parallel:false}
)
"""
graph.run(insertSSD)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
228,227583,21,227583,0,0,0,{},"{total: 228, errors: {}, committed: 228, failed: 0}","{total: 227583, errors: {}, committed: 227583, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 0, nodesDeleted: 0, nodesCreated: 0, labelsRemoved: 0, labelsAdded: 0, propertiesSet: 4321808}"


In [19]:
# Query 4: Creating Index on 'Name' for faster runtime
nameIndex = """
CREATE INDEX FOR (b:Book) ON (b.Name)
"""
graph.run(nameIndex)

(No data)

In [20]:
authorIndex = """
CREATE INDEX FOR (a:Author) ON (a.name)
"""
graph.run(authorIndex)

(No data)

In [21]:
reviewCountIndex = """
CREATE INDEX FOR (r:Review) ON (r.count)
"""
graph.run(reviewCountIndex)

(No data)

In [22]:
addUnstructuredProperties = """
CALL apoc.periodic.iterate(
  'MATCH (b:Book) RETURN b',
  'SET b.best_seller = false, b.in_1000_collection = false',
  {batchSize:1000, iterateList:true, parallel:false}
)
"""
graph.run(addUnstructuredProperties)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
702,701789,10,701789,0,0,0,{},"{total: 702, errors: {}, committed: 702, failed: 0}","{total: 701789, errors: {}, committed: 701789, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 0, nodesDeleted: 0, nodesCreated: 0, labelsRemoved: 0, labelsAdded: 0, propertiesSet: 1403578}"


In [23]:
duplicateRemoval = """
CALL apoc.periodic.iterate(
  'MATCH (b:Book)
   WITH b.Name AS title, collect(b) AS nodes 
   WHERE size(nodes) > 1
   RETURN nodes',
  'FOREACH (ignored IN tail($nodes) | DETACH DELETE ignored)',
  { batchSize: 100, parallel: false}
);
"""
graph.run(duplicateRemoval).data()

[{'batches': 448,
  'total': 44741,
  'timeTaken': 3,
  'committedOperations': 0,
  'failedOperations': 44741,
  'failedBatches': 448,
  'retries': 0,
  'errorMessages': {'Expected parameter(s): nodes': 448},
  'batch': {'total': 448,
   'errors': {'org.neo4j.graphdb.QueryExecutionException: Expected parameter(s): nodes': 448},
   'committed': 0,
   'failed': 448},
  'operations': {'total': 44741,
   'errors': {'Expected parameter(s): nodes': 448},
   'committed': 0,
   'failed': 44741},
  'wasTerminated': False,
  'failedParams': {},
  'updateStatistics': {'relationshipsDeleted': 0,
   'relationshipsCreated': 0,
   'nodesDeleted': 0,
   'nodesCreated': 0,
   'labelsRemoved': 0,
   'labelsAdded': 0,
   'propertiesSet': 0}}]

In [24]:
# Read data from 'topSellingBooks.txt'
with open('topSellingBooks.txt', 'r', encoding='utf-8') as file:
    top_selling_books = [line.strip() for line in file]

# Read data from 'thousandBooks.txt'
with open('1000books.txt', 'r', encoding='utf-8') as file:
    thousand_books = [line.strip() for line in file]
    
# Set values to true for specific conditions
bestSellerTrue = """
CALL apoc.periodic.iterate(
  'UNWIND $names AS name RETURN name',
  'MATCH (b:Book) WHERE b.Name CONTAINS name SET b.best_seller = true',
  {batchSize:1000, params:{names: $top_selling_books}}
)
"""

thousandBookTrue = """
CALL apoc.periodic.iterate(
  'UNWIND $names AS name RETURN name',
  'MATCH (b:Book) WHERE b.Name CONTAINS name SET b.in_1000_collection = true',
  {batchSize:1000, params:{names: $thousand_books}}
)
"""

# Execute the queries
graph.run(bestSellerTrue, top_selling_books=top_selling_books)
graph.run(thousandBookTrue, thousand_books=thousand_books)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
1,1000,104,1000,0,0,0,{},"{total: 1, errors: {}, committed: 1, failed: 0}","{total: 1000, errors: {}, committed: 1000, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 0, nodesDeleted: 0, nodesCreated: 0, labelsRemoved: 0, labelsAdded: 0, propertiesSet: 48944}"


In [25]:
# Query 9: Create WROTE relationships
wroteRelationship = """
CALL apoc.periodic.iterate(
    "MATCH (b:Book) UNWIND b.Authors AS author RETURN b, author",
    "MERGE (a:Author {name: author}) MERGE (a)-[:WROTE]->(b)",
    {batchSize:10000, parallel:false})
"""
graph.run(wroteRelationship)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
71,701789,10,701789,0,0,0,{},"{total: 71, errors: {}, committed: 71, failed: 0}","{total: 701789, errors: {}, committed: 701789, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 701789, nodesDeleted: 0, nodesCreated: 278663, labelsRemoved: 0, labelsAdded: 278663, propertiesSet: 278663}"


In [26]:
# Query 7: Create HAS_RATING relationships
hasRatingRelationship = """
CALL apoc.periodic.iterate(
  "MATCH (b:Book) WHERE b.average_rating IS NOT NULL RETURN b",
  "MERGE (r:Rating {value: b.average_rating}) MERGE (b)-[:HAS_RATING]->(r)",
  {batchSize: 10000, parallel: false}
);
"""
graph.run(hasRatingRelationship)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
228,227464,33,227464,0,0,0,{},"{total: 228, errors: {}, committed: 228, failed: 0}","{total: 227464, errors: {}, committed: 227464, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 227464, nodesDeleted: 0, nodesCreated: 304, labelsRemoved: 0, labelsAdded: 304, propertiesSet: 304}"


In [27]:
# Query 8: Create WRITTEN_IN relationships
writtenInRelationship = """
CALL apoc.periodic.iterate(
  "MATCH (b:Book) WHERE b.Language IS NOT NULL RETURN b",
  "MERGE (l:Language {name: b.Language}) MERGE (b)-[:WRITTEN_IN]->(l)",
  {batchSize:10000, parallel:false}
)
"""
graph.run(writtenInRelationship)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
139,138680,5,138680,0,0,0,{},"{total: 139, errors: {}, committed: 139, failed: 0}","{total: 138680, errors: {}, committed: 138680, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 138680, nodesDeleted: 0, nodesCreated: 81, labelsRemoved: 0, labelsAdded: 81, propertiesSet: 81}"


In [28]:
# Query 10: Create HAS_REVIEW relationships
hasReviewRelationship = """
CALL apoc.periodic.iterate(
  "MATCH (b:Book) RETURN b",
  "MERGE (r:Review {count: toInteger(b.CountsOfReview)}) MERGE (b)-[:HAS_REVIEW]->(r)",
  {batchSize:10000, parallel:false}
)
"""
graph.run(hasReviewRelationship)

batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
71,701789,6,701789,0,0,0,{},"{total: 71, errors: {}, committed: 71, failed: 0}","{total: 701789, errors: {}, committed: 701789, failed: 0}",false,{},"{relationshipsDeleted: 0, relationshipsCreated: 701789, nodesDeleted: 0, nodesCreated: 2679, labelsRemoved: 0, labelsAdded: 2679, propertiesSet: 2679}"


# Step 9: 5 Complex Queries 

### Query 1: Top Authors by Best Sellers
Retrieves the top 15 authors with the highest number of best-selling books, displaying their names, all distinct books they wrote, and the total count of books.

### Query 2: Authors with Diverse Language Writings
Identifies authors who have written books in multiple languages, showcasing their names, distinct languages used, and the total number of books written.

### Query 3: Authors with High Average Ratings
Selects authors whose books have an average rating calculated from reviews with more than 10,000 ratings. Displays author names, average ratings, and total ratings counts.

### Query 4: Best-Selling Books with Favorable Reviews
Returns a list of book titles marked as best sellers with high ratings and favorable reviews, based on a 5 star rating distribution of over 50%. 

### Query 5: Books with the Most Similar Titles
Retrieves titles of the top 75 books from the 'in_1000_collection' with the most similar titles, providing the count of similar titles and their respective work IDs.


In [29]:
# Query 1
query1 = """
MATCH (author:Author)-[:WROTE]->(book:Book)
WITH author, COLLECT(DISTINCT book.Name) AS AllBooks, ANY(b IN COLLECT(book) WHERE b.best_seller = true) AS hasBestSeller
WHERE hasBestSeller
RETURN author.name AS Author, AllBooks, SIZE(AllBooks) AS BookCount
ORDER BY BookCount DESC
LIMIT 15;
"""
result1 = graph.run(query1)
result1.to_data_frame()

,Author,AllBooks,BookCount
0,Anonymous,[Advice on the Art of Governance (Mau'izah-I J...,1159
1,Francine Pascal,"[Wired (Fearless, #33), Love & Betrayal & Hold...",622
2,Harold Bloom,"[Willa Cather (Bloom's Modern Critical Views),...",509
3,Carolyn Keene,[Dead on Arrival (Nancy Drew and the Hardy Boy...,442
4,Isaac Asimov,[Henry Hudson: Arctic Explorer and North Ameri...,425
5,Unknown,"[Beowulf, When Were You Born, People of India,...",400
6,R.L. Stine,"[The Beast from the East (Goosebumps, #43), Su...",397
7,Agatha Christie,[Dame Agatha Abroad: Murder on the Orient Expr...,311
8,Hal Leonard Publishing Company,"[Eric Clapton - From the Cradle, The Music Man...",298
9,NOT A BOOK,"[Red Covers Postcard Book, Radio Program: The...",243


In [30]:
# Query 2
query2 = """
MATCH (a:Author)-[:WROTE]->(b:Book)-[:WRITTEN_IN]->(l:Language)
WITH distinct(a), COLLECT(DISTINCT l.name) AS Languages, COUNT(*) AS NumberOfBooks
WHERE size(Languages) > 1
RETURN distinct(a.name) AS Author, Languages, NumberOfBooks
ORDER BY NumberOfBooks DESC
LIMIT 10;
"""
result2 = graph.run(query2)
result2.to_data_frame()

,Author,Languages,NumberOfBooks
0,Francine Pascal,"[eng, en-US, spa, en-GB]",412
1,Nora Roberts,"[eng, en-US, fre, spa, en-GB, ger]",373
2,Stephen King,"[eng, en-US, fre, spa, en-GB, en-CA, ger, ita,...",367
3,Anonymous,"[eng, en-US, fre, spa, mul, en-GB, grc, enm, j...",292
4,Agatha Christie,"[eng, en-US, fre, spa, en-GB, ger, ita]",290
5,William Shakespeare,"[eng, en-US, fre, spa, mul, en-GB, enm, ger]",268
6,R.L. Stine,"[eng, en-US, fre, spa, ger, ita]",252
7,Ann M. Martin,"[eng, en-US, fre, spa, en-GB]",248
8,Terry Pratchett,"[eng, en-US, fre, spa, en-GB, en-CA, ger]",196
9,Isaac Asimov,"[eng, en-US, fre, spa, en-GB, ger]",191


In [31]:
# Query 3
query3 = """
MATCH (a:Author)-[:WROTE]->(b:Book)-[:HAS_RATING]->(r:Rating)
WHERE toInteger(b.ratings_count) > 10000
WITH a, COLLECT({rating: toFloat(r.value), ratingsCount: toInteger(b.ratings_count)}) AS ratingsData
RETURN
a.name AS Author,
REDUCE(s = 0.0, r IN ratingsData | s + r.rating) / SIZE(ratingsData) AS avg_rating,
REDUCE(totalCount = 0, r IN ratingsData | totalCount + r.ratingsCount) AS total_ratings_count
ORDER BY avg_rating DESC
LIMIT 15;
"""
result3 = graph.run(query3)
result3.to_data_frame()

,Author,avg_rating,total_ratings_count
0,Bill Watterson,4.724286,167309
1,Anonymous,4.640000,56209
2,Gordon B. Hinckley,4.610000,12811
3,Alisa Kwitney,4.610000,11702
4,Gary Russell,4.590000,24419
5,Jude Fisher,4.590000,17516
6,J.K. Rowling,4.574615,5827771
7,Patrick Rothfuss,4.570000,253462
8,Sarah Young,4.540000,35891
9,Hiromu Arakawa,4.535000,131216


In [32]:
# Query 4
query4 = """
MATCH (b:Book)-[:HAS_REVIEW]->(r:Review)
WHERE toInteger(r.count) > 10000 AND toInteger(split(b.RatingDist5, ':')[1]) > 0.5 * toInteger(r.count)
RETURN b.Name AS Title, b.best_seller AS BestSeller
LIMIT 50;
"""
result4 = graph.run(query4)
result4.to_data_frame()

,Title,BestSeller
0,Harry Potter and the Half-Blood Prince (Harry ...,True
1,Harry Potter and the Order of the Phoenix (Har...,True
2,Harry Potter and the Sorcerer's Stone (Harry P...,False
3,Harry Potter and the Prisoner of Azkaban (Harr...,True
4,Harry Potter and the Goblet of Fire (Harry Pot...,True
5,The Fellowship of the Ring (The Lord of the Ri...,False
6,"Hatchet (Brian's Saga, #1)",False
7,Atlas Shrugged,False
8,Of Mice and Men,False
9,"Angels & Demons (Robert Langdon, #1)",True


In [37]:
# Query 5
query5 = """
MATCH (b:Book)
WHERE b.in_1000_collection = true
WITH b, size(split(coalesce(b.similar_books, ''), ',')) AS NumberOfSimilarBooks, split(coalesce(b.similar_books, ''), ',') AS SimilarBookWorkIds
OPTIONAL MATCH (b2:Book)
WHERE b2.work_id IN SimilarBookWorkIds
RETURN b.Name AS Title, NumberOfSimilarBooks, SimilarBookWorkIds
ORDER BY NumberOfSimilarBooks DESC
LIMIT 75;
"""
result5 = graph.run(query5)
result5.to_data_frame()


,Title,NumberOfSimilarBooks,SimilarBookWorkIds
0,A Short History of Nearly Everything,18,"[['373572', '6117055', '331227', '2018682',..."
1,Bryson's Dictionary of Troublesome Words: A Wr...,18,"[['216487', '700653', '275815', '138270', ..."
2,"Hatchet (Brian's Saga, #1)",18,"[['41667', '438131', '124245', '13164526', ..."
3,Guts: The True Stories behind Hatchet and the ...,18,"[['295419', '175089', '611726', '295617', ..."
4,The Heidi Chronicles: Uncommon Women and Other...,18,"[['205805', '413705', '784709', '170539', ..."
...,...,...,...
70,To the Lighthouse,18,"[['31072', '91494', '366524', '126583', '1..."
71,Mrs. Dalloway,18,"[['160010', '16810', '3102', '606805', '51..."
72,A Room of One's Own,18,"[['149709', '224387', '85767', '98532', '6..."
73,The Passion,18,"[['258627', '304157', '31186', '115009', '..."
